# Exploring Hacker News Posts

This project seeks to identify trends with Hacker News Posts. Hacker News is a site where user-submitted stories (a.k.a. posts) are voited and commented upon. We will be using an abridged version of the following dataset: https://www.kaggle.com/hacker-news/hacker-news-posts. The original version of the dataset has about 300,000 rows. We are using a dataset that is a random sample of 20,000 submissions. The 280,000 rows were removed because they did not receive any comments.

We will be looking at 2 main types of posts: Ask HN and Show HN. Ask HN posts are submitted when a user has a specific question they wish to ask the community. Show HN posts are submitted when a user would like to showcase a project, product, or some other interesting content to the community.

We will compare these two types of posts to determine:
1. Do Ask HN or Show HN receive more comments on average?
2. Do posts created at a certain time receive more comments on average?

## Importing the Data

In [1]:
# Put all imports here
from csv import reader
import datetime as dt

In [2]:
with open("hacker_news.csv") as hn_file:
    hn_content = reader(hn_file)
    hn = list(hn_content)
    
hn[:5]

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

## Removing the Header

We can see that in our hn object, the first item is the header. Let's remove the header and store in a variable so we can analyze our data.

In [3]:
headers = hn[0]
hn = hn[1:]

print("Header is:", headers)
print("\n")
print("First five rows are:", hn[:5])

Header is: ['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


First five rows are: [['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '

## Extracting only Ask HN and Show HN Posts

In [4]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    lower_title = row[1].lower()
    if lower_title.startswith('ask hn'):
        ask_posts.append(row)
    elif lower_title.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print("Number of posts in ask_posts:", len(ask_posts))
print("Number of posts in show_posts:", len(show_posts))
print("Number of posts in other_posts:", len(other_posts))

Number of posts in ask_posts: 1744
Number of posts in show_posts: 1162
Number of posts in other_posts: 17194


## Average Number of Comments for Ask HN and Show HN Posts

In [5]:
total_ask_comments = 0
for row in ask_posts:
    total_ask_comments += float(row[4])
    
total_show_comments = 0
for row in show_posts:
    total_show_comments += float(row[4])
    
avg_ask_comments = total_ask_comments / len(ask_posts)
avg_show_comments = total_show_comments / len(show_posts)

print("Average number of comments for an Ask HN post:", avg_ask_comments)
print("Average number of comments for a Show HN post:", avg_show_comments)

Average number of comments for an Ask HN post: 14.038417431192661
Average number of comments for a Show HN post: 10.31669535283993


Based on the above findings, it looks like Ask HN posts receive almost 4 more comments on average. Ask HN posts on average receive 14.04 comments, while Show HN posts on average receive 10.32 comments.

Because of this, we will focus the rest of this project on Ask posts. Specifically, we will determine how the time of a post affects the number of comments it receives.

## Finding the Amount of Ask Posts and Comments by Hour Created

We will perform these two steps:
1. Calculate the number of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments each ask post receives by hour created.

### Calculating Number of Ask Posts and Comments Per Hour

In [6]:
result_list = []
for post in ask_posts:
    created_at = dt.datetime.strptime(post[6], "%m/%d/%Y %H:%M")
    num_comments = int(post[4])
    result_list.append([created_at, num_comments])
    
# This will give us the number of ASK HN posts by hour.    
counts_by_hour = {}
# This will give us the number of comments on ASK HN posts by hour.
comments_by_hour = {}

for row in result_list:
    hour = row[0].hour
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
    else:
        counts_by_hour[hour] += 1
    
    if hour not in comments_by_hour:
        comments_by_hour[hour] = row[1]
    else:
        comments_by_hour[hour] += row[1]
        
print("Posts By Hour:", counts_by_hour)
print("\n")
print("Comments By Hour:", comments_by_hour)

Posts By Hour: {9: 45, 13: 85, 10: 59, 14: 107, 16: 108, 23: 68, 12: 73, 17: 100, 15: 116, 21: 109, 20: 80, 2: 58, 18: 109, 3: 54, 5: 46, 19: 110, 1: 60, 22: 71, 8: 48, 4: 47, 0: 55, 6: 44, 7: 34, 11: 58}


Comments By Hour: {9: 251, 13: 1253, 10: 793, 14: 1416, 16: 1814, 23: 543, 12: 687, 17: 1146, 15: 4477, 21: 1745, 20: 1722, 2: 1381, 18: 1439, 3: 421, 5: 464, 19: 1188, 1: 683, 22: 479, 8: 492, 4: 337, 0: 447, 6: 397, 7: 267, 11: 641}
